In [2]:
import cv2
import pandas as pd 
import numpy as np
import os 
import matplotlib as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense,
    Flatten, Input
)

In [ ]:
label=pd.read_csv("labels.csv")
classid=label['ClassId'].tolist()
labelname=label['Name'].tolist()
number_of_classes=len(label)


In [21]:
import os
import cv2
import numpy as np

data = []
labels = []
image_size = 32  # standard for traffic signs

dataset_path = "dataset/DATA"
class_names = os.listdir(dataset_path)  # e.g., ['Stop', 'SpeedLimit', 'TurnLeft']
label_namesid= [int(i) for i in class_names]


for label, class_name in zip(label_namesid,class_names):
    class_folder = os.path.join(dataset_path, class_name)
    print(class_folder)
    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue  # skip unreadable images

        img = cv2.resize(img, (image_size, image_size))
        data.append(img)
        labels.append(label)

X = np.array(data)
y = np.array(labels)

print("Loaded images:", X.shape)
print("Labels:", y.shape)


dataset/DATA\0
dataset/DATA\1
dataset/DATA\10
dataset/DATA\11
dataset/DATA\12
dataset/DATA\13
dataset/DATA\14
dataset/DATA\15
dataset/DATA\16
dataset/DATA\17
dataset/DATA\18
dataset/DATA\19
dataset/DATA\2
dataset/DATA\20
dataset/DATA\21
dataset/DATA\22
dataset/DATA\23
dataset/DATA\24
dataset/DATA\25
dataset/DATA\26
dataset/DATA\27
dataset/DATA\28
dataset/DATA\29
dataset/DATA\3
dataset/DATA\30
dataset/DATA\31
dataset/DATA\32
dataset/DATA\33
dataset/DATA\34
dataset/DATA\35
dataset/DATA\36
dataset/DATA\37
dataset/DATA\38
dataset/DATA\39
dataset/DATA\4
dataset/DATA\40
dataset/DATA\41
dataset/DATA\42
dataset/DATA\43
dataset/DATA\44
dataset/DATA\45
dataset/DATA\46
dataset/DATA\47
dataset/DATA\48
dataset/DATA\49
dataset/DATA\5
dataset/DATA\50
dataset/DATA\51
dataset/DATA\52
dataset/DATA\53
dataset/DATA\54
dataset/DATA\55
dataset/DATA\56
dataset/DATA\57
dataset/DATA\6
dataset/DATA\7
dataset/DATA\8
dataset/DATA\9
Loaded images: (4170, 32, 32, 3)
Labels: (4170,)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = X / 255.0  # Normalize pixel values
y = to_categorical(y)  # One-hot encode class labels

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(number_of_classes, activation='softmax')  # dynamic output layer
])


In [26]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [28]:
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/20
53/53 [==============================] - 3s 51ms/step - loss: 0.3452 - accuracy: 0.8951 - val_loss: 0.2961 - val_accuracy: 0.9017
Epoch 2/20
53/53 [==============================] - 3s 52ms/step - loss: 0.3099 - accuracy: 0.9014 - val_loss: 0.2009 - val_accuracy: 0.9400
Epoch 3/20
53/53 [==============================] - 3s 53ms/step - loss: 0.2582 - accuracy: 0.9203 - val_loss: 0.1725 - val_accuracy: 0.9436
Epoch 4/20
53/53 [==============================] - 3s 51ms/step - loss: 0.2186 - accuracy: 0.9326 - val_loss: 0.1558 - val_accuracy: 0.9472
Epoch 5/20
53/53 [==============================] - 3s 54ms/step - loss: 0.2033 - accuracy: 0.9379 - val_loss: 0.1522 - val_accuracy: 0.9580
Epoch 6/20
53/53 [==============================] - 3s 55ms/step - loss: 0.1612 - accuracy: 0.9508 - val_loss: 0.0984 - val_accuracy: 0.9736
Epoch 7/20
53/53 [==============================] - 3s 52ms/step - loss: 0.1363 - accuracy: 0.9628 - val_loss: 0.0877 - val_accuracy: 0.9712
Epoch 8/20
53

In [31]:
model.save("traffic_sign_model.h5")


In [33]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

# Load saved model
model = load_model("traffic_sign_model.h5")

# Load class names (order should match training)
class_names = sorted(os.listdir("dataset"))  # adjust path if needed


In [36]:

img_path = "test1.png"  # your image path here

# Load and preprocess the image
img = cv2.imread(img_path)
img = cv2.resize(img, (32, 32))        # match training size
img_normalized = img / 255.0           # normalize like training
img_input = img_normalized.reshape(1, 32, 32, 3)  # reshape for model


In [40]:
prediction = model.predict(img_input)
predicted_class = np.argmax(prediction)
confidence = np.max(prediction)


print(f"Predicted Class: {predicted_class} ({confidence * 100:.2f}%)")
cv2.putText(img, f"{label} ({confidence*100:.1f}%)", (5, 25),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

cv2.imshow("Prediction", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 25ms/step
Predicted Class: 0 (98.30%)
